<a href="https://colab.research.google.com/github/m-zayed5722/Misc_Projects/blob/main/Cartridge_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, DiceLoss, JaccardLoss
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose

# 1: Data Acquisition and Preprocessing

In [ ]:
import os
import cv2


# Assuming image and mask filenames are related (e.g., 'image_001.jpg' and 'mask_001.jpg')
# Load images and masks using OpenCV
images = []
masks = []

# Convert lists to NumPy arrays for further processing
images = np.array(images)
masks = np.array(masks)

In [ ]:
# Preprocessing: split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)

# Further split for validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)


In [ ]:

# Generate augmented images and masks using the flow method
augmented_data = datagen.flow(np.array(X_train), np.array(y_train), batch_size=len(X_train), shuffle=False)

# Extract augmented images and masks from the generated batches
augmented_images, augmented_masks = augmented_data.next()

# Update training set with augmented data
X_train = np.concatenate([X_train, augmented_images])
y_train = np.concatenate([y_train, augmented_masks])

In [ ]:
# Display the first few original and augmented images and masks
#...

#2: Implement the U-net model and training process

In [ ]:
# Define U-Net architecture for 2D images
def u_net_2d(input_size):
    inputs_2d = keras.Input(input_size)

    # Encoder
    conv1_2d = Conv2D(32, (3, 3), activation='relu')(inputs_2d)
    pool1_2d = MaxPooling2D((2, 2))(conv1_2d)

    # Downsampling
    conv2_2d = Conv2D(64, (1, 1), activation='elu', padding='same', dilation_rate=1)(pool1_2d)
    pool2_2d = MaxPooling1D(2)(conv2_2d)

    conv3_2d = Conv2D(128, (1, 1), activation="elu", padding='same', dilation_rate=2)(pool2_2d)
    pool3_2d = MaxPooling2D()(conv3_2d)

    # Bottleneck
    conv4_2d = Conv2D(256, (1, 1), activation="elu", padding='same')(pool3_2d)

    up1_2d = UpSampling2D((2, 2))(conv4_2d)
    concat1_2d = concatenate([up1_2d, conv3_2d])
    conv5_2d = Conv2D(128, (1, 1), activation="elu")(concat1_2d)

    up2_2d = UpSampling2D((2, 2))(conv5_2d)
    concat2_2d = concatenate([up2_2d, conv2_2d])
    conv6_2d = Conv2D(64, (1, 1), activation="elu")(concat2_2d)

    outputs_2d = conv6_2d

    u_net_2d_model = Model(inputs=[inputs_2d], outputs=[outputs_2d])
    return u_net_2d_model

In [ ]:
# Define a function to train the U-Net
def train_u_net(train_data_dir, train_mask_dir, validation_data_dir=None, validation_mask_dir=None):
    # Load and preprocess the data
    train_img_generator = ImageDataGenerator(rescale=1./255)
    train_mask_generator = ImageDataGenerator(rescale=1./255)

    # Training data generator parameters
    train_data_gen_args = dict(rotation_range=90, width_shift_range=0.1, height_shift_range=0.1, shear_range=0, zoom_range=0)

    # Compile the model
    model = u_net_2d(input_size=(height, width, channels))
    model.summary()
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

    steps_per_epoch = 10
    epochs = 100

    if validation_data_dir:
        val_img_generator = ImageDataGenerator(rescale=1./255)
        val_mask_generator = ImageDataGenerator(rescale=1./255)
        validation_data_gen_args = dict()
        val_data_generator = val_img_generator.flow_from_directory(
            validation_data_dir,
            class_mode=None,
            color_mode='rgb',
            target_size=(height, width),
            batch_size=batch_size,
            seed=42
        )
        validation_steps = len(val_data_generator)
    else:
        validation_steps = None

    history = model.fit(
        train_data_generator(train_data_dir, train_mask_dir, train_img_generator, train_mask_generator),
        validation_data=val_data_generator if validation_data_dir else None,
        validation_steps=validation_steps,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch
    )
    return history


In [ ]:
# Train U-Net
history = train_u_net(X_train, y_train, validation_data=(X_val, y_val))


#3: Hyperparameter Optimization

In [ ]:
# Define the U-Net architecture for hyperparameter optimization
def u_net_hyperparameter_optimization(input_size, batch_size, loss_function, optimizer):
    inputs = Input(input_size)

    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    pool3 = MaxPooling2D((2, 2))(conv3)

    # Bottleneck
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)

    # Decoder
    up5 = UpSampling2D((2, 2))(conv4)
    concat5 = concatenate([up5, conv3], axis=3)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(concat5)

    up6 = UpSampling2D((2, 2))(conv5)
    concat6 = concatenate([up6, conv2], axis=3)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat6)

    up7 = UpSampling2D((2, 2))(conv6)
    concat7 = concatenate([up7, conv1], axis=3)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat7)

    # Output layer
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv7)

    # Compile the model with the specified hyperparameters
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

    return model



In [ ]:

# Define the hyperparameters to be evaluated
param_grid = {
    'batch_size': [8, 16, 32, 64],
    'loss_function': [BinaryCrossentropy(), DiceLoss(), JaccardLoss()],
    'optimizer': [Adam()]
}


In [ ]:
# Perform grid search for hyperparameter optimization
grid_search = GridSearchCV(estimator=u_net_hyperparameter_optimization, param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)


In [ ]:
# Get the best hyperparameters
best_hyperparameters = grid_search.best_params_
print("Best hyperparameters:", best_hyperparameters)

#4: Model Optimization and Fine-Tuning

In [ ]:
# Define the U-Net architecture with the best hyperparameters
best_model = unet_model_for_hp_optimization(input_size=(256, 256, 3), num_filters=best_num_filters, dropout_rate=best_dropout_rate)



In [ ]:
# Compile the model
best_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
# Train the model with the best hyperparameters
history = best_model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val, y_val))



In [ ]:
# Optional: Visualize training history
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:

# Evaluate the model on the test set
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')



In [ ]:
# Additional Fine-Tuning and Optimization Steps
# ...

# Save the optimized model for future use
best_model.save('optimized_model.h5')


#5: Result Analysis and Application

In [ ]:
# Load the optimized model
optimized_model = tf.keras.models.load_model('optimized_model.h5')



In [ ]:
# Evaluate the optimized model on the test set
test_loss, test_accuracy = optimized_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')



In [ ]:
# Optionally, make predictions on the test set
predictions = optimized_model.predict(X_test)



In [ ]:
# Perform post-processing on predictions if needed
# ...

# Visualize some test examples with predicted masks
visualize_test_examples(X_test, predictions, y_test)


In [ ]:

# Real-world application
# ...

# Save the final results or predictions for future reference
# ...

def visualize_test_examples(X_test, predictions, y_test):
    # Implement a visualization function based on your needs
    # Display some test examples with the original images, ground truth masks, and predicted masks
    # You can use a library like matplotlib for visualization
    # ...

# Additional Result Analysis and Application Steps
# ...

# Save final results or predictions for future reference
# ...

# Real-world application
# ...
